# **Python + MySQL**

Hoje veremos como é feita a ligação entre o python e o MySQL.

Vale lembrar que se trata de um CRUD de DML (Data Manipulation Language) e por tanto, as tabelas e suas respectivas colunas devem estar criadas.

Primeiramente vamos instalar o MySQL Connector que é uma biblioteca responsável por conectar-se e fazer modificações no banco de dados.

Para tanto, execute o comando no terminal:

![mysqlconnector](img/mysqlconnector.png)

Após isso iremos criarum arquivo chamado **db_connector.py** que será responsável pela conexão com o banco:

In [ ]:
import mysql.connector

class DatabaseConnection:
    def __init__(self, config:dict):
        self.config = config
        self.connection = None

No código anterior, importamos a biblioteca mysql.connector e vamos usar a orientação a objetos em python para melhor organizar nosso código.

Note que usei apenas um atributo a ser recebido na nossa classe que é **config** esse atributo será um dicionário com as credenciais para acessar o banco de dados.

In [ ]:
config = {
    'user': '',
    'password': '',
    'host': '',
    'database': '',
    'port': 3306
}

Já o segundo atributo e a nossa conexão e ela será alterada de "estado" pelos nossos métodos a seguir:

In [ ]:
    def __enter__(self):
        self.connection = mysql.connector.connect(**self.config)
        return self.connection
    def __exit__(self, exc_type, exc_value, traceback):
        if self.connection:
            self.connection.close()

Esse 'dundder' __ enter __ fas parte do **protocolo de gerenciamento de contexto** e é usado em conjunto com a instrução "with".

Quando implementamos o método citado, estamos definindo o que acontece ao entrar-mos no bloco "with". E isso garante que:

- **Abertura e fechamento automático da conexão:** O método __ enter __ é chamado quando você entra no bloco with. Nesse método, você estabelece a conexão com o banco de dados. No final do bloco with, o método __exit__ é chamado, permitindo que você feche automaticamente a conexão. Isso ajuda a garantir que recursos, como conexões de banco de dados, sejam liberados corretamente, mesmo se ocorrerem exceções.

- **Gerenciamento automático de exceções:** O método __ exit __ recebe informações sobre exceções (se ocorrerem) como argumentos. Isso permite que você implemente lógica para lidar com exceções, como fazer rollback em transações não confirmadas, antes de fechar a conexão.

In [ ]:
    def __exit__(self, exc_type, exc_value, traceback):
        if self.connection:
            self.connection.close()

Agora vamos criar outro arquivo chamado **db_helper.py** que vai ser nosso executor de sódigo QUERY no nosso banco de dados.

Primeiro importamos o módulo **db_connector** e usamos a orientação a objetos em python para organizar nosso código:

In [ ]:
from db_connector import DatabaseConnection, config

class DBHelper():
    def __init__(self):
        self.connection = None

Note que nosso executor tem como argumento apenas o estado atual da conexão, sendo que os parâmetros de acesso ao banco já foram 'setados' na classe **DatabaseConnection**.

Agora vamos instruir no código a execução de uma QUERY ao nosso banco de dados:

In [ ]:
    def execute(self, sql):
        with DatabaseConnection(config) as connection: #Necessário ao criar método enter.
            cursor = connection.cursor()
            cursor.execute(sql)
            if sql.split()[0].upper() =='SELECT':
                result = list()#ok
                for row in cursor.fetchall():
                    result.append(row)
                connection.commit()
                cursor.close()
                return result
            else:
                connection.commit()
                cursor.close()
                return None

No código acima temos uma funcção chamada **execute** com apenas um parâmetro **sql** que vai ser a 'query' a ser executada. No corpo da função usamos a instrução **"with"** que foi descrita quando instruimos o gerenciamento de estado com o 'dundder' __ enter __.

Nela instanciamos a conexão com o banco de dados com o argumento **config** que são as chaves de acesso ao banco, e utilizamos o 'apelido' **connection** para essa instancia.

Depois criamos o **cursor** que é o responsável por **"selecionar"** o que vai acontecer dentro do banco.

E então executamos com o método **execute** o parâmetro sql.

Agora teremos duas situações possíveis.

Ou o banco retorna alguma coisa, ou não retorna nada.

A única querry que retorna algo é a **SELECT** e para ela criamos a condicional 'if' e nela recebemos uma string do nosso parametro 'sql' e separamos as palavras dessa string com o método "split" e pegamos a primeira palavra separada. 
Se ela for "SELECT" então tratamos o retorno em uma lista, e encerramos  a conecção com a conexão e com o cursor.
Se ela não for "SELECT" então fazemos o **"salvamento"** sem retornar nada.

## **Criando as funções de CRUD**

Temos com sucesso os métodos de acesso e execução das nossas "querys", agora vamos criar as funções CRUD para o banco de dados.

### **Estrutura básica:**

Precisamos seguir a estrutura básica para inserir os dados no banco, que é:
- Instanciar noddo **DBHelper** que vai conectar e executar a query.
- Solicitar qual query será executada
- Executar a query.

In [ ]:
def inserirTarefa(): #CREATE
    print()
    conn = DBHelper()
    tarefa = input("Digite a tarefa a ser inserida: ")
    sql = f"INSERT INTO ew_todolist (id, tarefa, feito) VALUES (Null, '{tarefa}', 1)"
    conn.execute(sql)

def listarTarefas(): # READ
    print()
    conn = DBHelper()
    tarefas = conn.execute("SELECT * FROM ew_todolist")
    for task in tarefas:
        if task[2]==0:
            print(f"\x1b[9mId:{task[0]} | {task[1]} | FEITO \x1b[0m")
        else:print(f"Id:{task[0]} | {task[1]} ")

def atualizarTarefa(): # UPDATE
    print()
    conn= DBHelper()
    listarTarefas()
    print()
    id_tarefa = int(input("Digite o número da tarefa que deseja atualizar: "))
    tarefa = input("Digite a tarefa atualizada: ")
    sql = conn.execute(f"UPDATE ew_todolist SET tarefa={tarefa} WHERE id={id_tarefa}") 
    print()
    listarTarefas()

def atualizarStatusTarefa(): # UPDATE
    print()
    conn= DBHelper()
    listarTarefas()
    print()
    id_tarefa = int(input("Digite o número da tarefa que foi concluída: "))
    sql = conn.execute(f"UPDATE ew_todolist SET feito=0 WHERE id={id_tarefa}") 
    print()
    listarTarefas()
   
def deletarTarefa(): # DELETE
    print()
    conn= DBHelper()
    listarTarefas()
    print()
    id_tarefa = int(input("Digite o número da tarefa que deseja deletar: "))
    sql = conn.execute(f"DELETE FROM ew_todolist WHERE id={id_tarefa}") 
    print()
    listarTarefas()